In [3]:
from pandas import Series, DataFrame

In [4]:
import pandas as pd

In [5]:
import numpy as np

In [6]:
import csv

In [7]:
from IPython.display import display, HTML

In [8]:
import string

In [9]:
import re

In [36]:
df_com_2016 = pd.read_excel('../com_booking_mapped2016.xlsx')

In [55]:
df_com_mapping.loc[:0]

,customer_name,unique_name,mapable,mapped_name,gaining_sub_scms,remarks,where_to,gaining_sales_level_2,gaining_sales_level_3,gaining_sales_level_4,gaining_sales_level_5,gaining_sales_level_6
0,GUAVUS NETWORK SYSTEMS PVT LTD,GUAVUS INDIA,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
df_com_mapping = pd.read_excel('../com_mapping_list_working.xlsx')

In [95]:
df_temp = df_com_mapping[df_com_mapping.mapped_name.str.contains('PEPSICO INDIA HOLDINGS PRIVATE LIMITED', case=False).fillna(False)]
df_temp = df_temp[['mapable', 'mapped_name', 'gaining_sub_scms', 'remarks','where_to','gaining_sales_level_2','gaining_sales_level_3','gaining_sales_level_4','gaining_sales_level_5','gaining_sales_level_6']]
mapped_name = df_temp.mapped_name.unique()
mapped_name[0]

u'PEPSICO INDIA HOLDINGS PRIVATE LIMITED'

In [10]:
def extract_sonos(cell_value):
    pattern = re.compile(r"[\(,\s]*(\d{5,})[\),\s]")
    match = pattern.findall(cell_value)
    return match

def get_sos_file_to_dict(file_names):
    data_dict = {}
    for filename in file_names:
        df = pd.read_excel('../'+filename)
        for index, row in df.iterrows():
            temp_dict = {row[0] : row[2]}
            data_dict.update(temp_dict)
    return data_dict
    
def get_sos_file_to_dict_by_pattern(file_name):
    data_dict = {}
    df = pd.read_excel('../'+file_name)
    df.rename(columns=string.lower, inplace=True)
    for index, row in df.iterrows():
        if row['remarks'] != '' and pd.notnull(row['remarks']):
            cus_name = row[0]
            so_nos = extract_sonos(row['remarks'])
            temp_dict = {so_no: cus_name for so_no in so_nos}
    return data_dict
    
def get_so_nos_from_file():
    data_dict = {}
    # Get SO_Nos from Vipul's update file
    data_dict.update(get_sos_file_to_dict_by_pattern('vipul_feedback.xlsx'))
    
    # Get ADP/Fidelity/Amway so numbers into Dataframe
    filenames = ['adp_sos.xlsx','fidelity_sos.xlsx','amway_sos.xlsx']
    data_dict.update(get_sos_file_to_dict(filenames))

    df_final = DataFrame.from_dict(data_dict, orient='index') # Dictionary to Dataframe will not have column names
    df_final.columns = ['mapped_name'] # Assign column names
    df_final['sales_order_number_detail'] = df_final.index # Make Index as column
    # Add More columns to the existing df_final dataframe
    empty_columns = ['mapped_name','mapable', 'gaining_sub_scms', 'remarks','where_to','gaining_sales_level_2','gaining_sales_level_3','gaining_sales_level_4','gaining_sales_level_5','gaining_sales_level_6']
    df_empty = pd.DataFrame(columns=empty_columns)
    
    df_final = pd.concat([df_final, df_empty]) # Concatenate df_final and df_empty to make df_final with all reqrd. columns
    srs_mapped_name = df_final['mapped_name'].unique() # Assign all unique customer_names from df_final into a list
    
    # Iterate over customers list, obtain other fields data from mapping list dataframe and assign them in df_final
    rqd_fields = ['mapable', 'mapped_name', 'gaining_sub_scms', 'remarks','where_to','gaining_sales_level_2','gaining_sales_level_3','gaining_sales_level_4','gaining_sales_level_5','gaining_sales_level_6']
    for name in srs_mapped_name:
        df_temp = df_com_mapping[df_com_mapping.mapped_name.str.contains(name, case=False).fillna(False)]
        if not df_temp.empty:
            df_temp = df_temp[rqd_fields]
            rqd_data = [df_temp.mapable.unique()[0],df_temp.mapped_name.unique()[0],df_temp.gaining_sub_scms.unique()[0],df_temp.remarks.unique()[0],df_temp.where_to.unique()[0],df_temp.gaining_sales_level_2.unique()[0],df_temp.gaining_sales_level_3.unique()[0],df_temp.gaining_sales_level_4.unique()[0],df_temp.gaining_sales_level_5.unique()[0],df_temp.gaining_sales_level_6.unique()[0]]
            df_final.ix[df_final.mapped_name.str.contains(name, case=False).fillna(False),rqd_fields] = rqd_data
    
    # Resetting and droping Index being sales order number and keep normal index
    df_final.reset_index(inplace=True) 
    df_final.drop(['index'], inplace=True, axis=1)
    df_final['sales_order_number_detail'] = df_final['sales_order_number_detail'].astype(int)
    return df_final

get_so_nos_from_file()


#extract_sonos(df_vipul.remarks[0])

NameError: global name 'df_com_mapping' is not defined